In [1]:
!pip install -q einops
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q transformers
!pip install -q peft
!pip install -q gradio
!apt install -q libomp-dev
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 

In [10]:
import torch

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)

import bs4
import requests
from typing import List

import nltk
from nltk import sent_tokenize

from tqdm import tqdm

import numpy as np

import torch

import faiss

import re

import transformers
from transformers import AutoModel, AutoTokenizer

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
base_model_id = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                bnb_4bit_use_double_quant=True)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "yurezsml/phi2_chan")

adapter_config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Подготовка текста с информацией про персонажа для улучшения фактологической связности

In [21]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def preprocess(text):
    text = text.lower()
    temp = remove_accents(text)
    text = text.replace('\xa0', ' ')
    text = text.replace('\n\n', '\n')
    text = text.replace('()', '')
    text = text.replace('[]', '')
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = text.replace('а́', 'а')
    return text

def split_text(text: str, n=2, character=" ") -> List[str]:
    text = preprocess(text)

    all_sentences = sent_tokenize(text)
    return [' '.join(all_sentences[i : i + n]) for i in range(0, len(all_sentences), 2)]


def split_documents(documents: List[str]) -> list:
    texts = []
    for text in documents:
        if text is not None:
            for passage in split_text(text):
                texts.append(passage)

    return texts


def embed(text, model, tokenizer):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [11]:
response = requests.get("https://en.wikipedia.org/wiki/Chandler_Bing")

base_text = ''

if response:
    html = bs4.BeautifulSoup(response.text, 'html.parser')

    title = html.select("#firstHeading")[0].text
    paragraphs = html.select("p")
    for para in paragraphs:
        base_text = base_text + para.text

fact_coh_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/bert-base-multilingual-cased-sentence")
fact_coh_model = AutoModel.from_pretrained("DeepPavlov/bert-base-multilingual-cased-sentence")
fact_coh_model.to(device)

nltk.download('punkt')
subsample_documents = split_documents([base_text])

batch_size = 8
total_batches = len(subsample_documents) // batch_size + (0 if len(subsample_documents) % batch_size == 0 else 1)

base = list()
for i in tqdm(range(0, len(subsample_documents), batch_size), total=total_batches, desc="Processing Batches"):
  batch_texts = subsample_documents[i:i + batch_size]
  base.extend(embed(batch_texts, fact_coh_model, fact_coh_tokenizer))

base = np.array([vector.cpu().numpy() for vector in base])

index = faiss.IndexFlatL2(base.shape[1])
index.add(base)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Processing Batches: 100%|██████████| 6/6 [00:02<00:00,  2.23it/s]


In [12]:
import gradio as gr
import asyncio

In [57]:
async def get_context(subsample_documents, query, index, model, tokenizer):
  k = 5
  xq = embed(query.lower(), model, tokenizer).cpu().numpy()
  D, I = index.search(xq.reshape(1, 768), k)
  return subsample_documents[I[0][0]]

async def get_prompt(question, use_rag, answers_history: list[str]):
  eval_prompt = '###system: answer the question as Chandler. '
  for idx, text in enumerate(answers_history):
    if idx % 2 == 0:
      eval_prompt = eval_prompt + f' ###question: {text}'
    else:
      eval_prompt = eval_prompt + f' ###answer: {text} '
  if use_rag:
    context = await asyncio.wait_for(get_context(subsample_documents, question, index, fact_coh_model, fact_coh_tokenizer), timeout=60)
    eval_prompt = eval_prompt + f' Chandler. {context}'
  eval_prompt = eval_prompt + f' ###question: {question} '
  eval_prompt = ' '.join(eval_prompt.split())
  return eval_prompt

In [66]:
async def get_answer(question, use_rag, answers_history: list[str]):
  eval_prompt = await asyncio.wait_for(get_prompt(question, use_rag, answers_history), timeout=60)
  model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)
  ft_model.eval()
  with torch.no_grad():
    answer = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=30, repetition_penalty=1.11)[0], skip_special_tokens=True) + '\n'
  answer = ' '.join(answer.split())
  if eval_prompt in answer:
    answer = answer.replace(eval_prompt,'')
  answer = answer.split('###answer')[1]

  dialog = ''
  for idx, text in enumerate(answers_history):
    if idx % 2 == 0:
      dialog = dialog + f'you: {text}\n'
    else:
      dialog = dialog + f'Chandler: {text}\n'
  dialog = dialog + f'you: {question}\n'
  dialog = dialog + f'Chandler: {answer}\n'

  answers_history.append(question)
  answers_history.append(answer)

  return dialog, answers_history

In [67]:
async def async_proc(question, use_rag, answers_history: list[str]):
  try:
    return await asyncio.wait_for(get_answer(question, use_rag, answers_history), timeout=60)
  except asyncio.TimeoutError:
    return "Processing timed out.", answers_history

In [69]:
gr.Interface(
    fn=async_proc,
    inputs=[
        gr.Textbox(
            label="Question",
        ),
        gr.Checkbox(label="Use RAG", info="Pick to RAG to improve factual coherence"),
        gr.State(value=[]),
    ],
    outputs=[
        gr.Textbox(
            label="Chat"
        ),
        gr.State(),
    ],
    title="Асинхронный сервис для чат-бота по сериалу Друзья",
    concurrency_limit=5
).queue().launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8259b73f73341738a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8259b73f73341738a5.gradio.live
